In [21]:
#import necessary packages
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
    
import time
from bs4 import BeautifulSoup
from IPython.display import display_html

import pandas as pd

In [65]:
def downloadDSEArchiveData(dataUrl, fromDay, fromMonth, fromYear, toDay, toMonth, toYear, instrument):
    # chromedriver.exe path should be added in environment variables
    #import os
    #for k, v in os.environ.items():
    #    print(f'{k}={v}')
    
    # load/install the chrome-driver
    driver = webdriver.Chrome()
    
    # load data archive website
    driver.get(dataUrl)
    
    print('URL:' + driver.current_url)
    
    try:
        alert = driver.switch_to.alert
        alert.accept()
    except NoAlertPresentException:
        print("No alert displayed!")
        
    #Calender behavior may varies.Observe calender behavior in your brwoser and send keys
    driver.implicitly_wait(2)
    startDate = driver.find_element_by_id('DayEndSumDate1')
    startDate.send_keys(fromDay)
    startDate.send_keys(fromMonth)
    startDate.send_keys(Keys.TAB)
    startDate.send_keys(fromYear)

    driver.implicitly_wait(2)
    endDate = driver.find_element_by_id('DayEndSumDate2')
    endDate.send_keys(toDay)
    endDate.send_keys(toMonth)
    endDate.send_keys(Keys.TAB)
    endDate.send_keys(toYear)

    # if you want to download data for a specific instrument
    inst = Select(driver.find_element_by_name("Symbol"))
    inst.select_by_visible_text(instrument)

    # generate the table
    driver.find_element_by_name("ViewDayEndArchive").click()

    driver.implicitly_wait(120)
    
    # extract the table
    soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table")
    dataset = pd.read_html(driver.page_source, header=0)[len(tables) - 5]
    
    #print(dataset)
    dataset = dataset.drop(["#"], axis=1)

    # write the file to csv
    filename = instrument + "_" + fromDay + "-" + fromMonth + "-" + fromYear + "_" + toDay + "-" + toMonth + "-" + toYear + ".csv"
    dataset.to_csv(filename, index=False)

    return dataset

In [67]:
dataUrl = "https://www.dsebd.org/day_end_archive.php"
driverUrl = "E:\Projects\Data Science\Stock Analysis\chromedriver_win32"

fromDay = "16"
fromMonth = "Apr"
fromYear = "2018"

toDay = "15"
toMonth = "Mar"
toYear = "2020"

instrument = "RENATA"

dataset = downloadDSEArchiveData(dataUrl, fromDay, fromMonth, fromYear, toDay, toMonth, toYear, instrument)

print(dataset.shape)

dataset.describe()

URL:https://www.dsebd.org/data_archive.php
No alert displayed!
(466, 11)


,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
count,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000
mean,1181.867811,1190.222532,1174.027468,1182.149785,1189.133691,1189.585193,254.319742,13.335167,11124.439914
std,115.638846,115.496890,114.816556,115.008115,66.156628,65.759024,191.787558,17.021124,13667.455249
min,0.000000,0.000000,0.000000,0.000000,982.800000,982.800000,0.000000,0.000000,0.000000
25%,1146.275000,1151.550000,1141.275000,1148.600000,1146.625000,1146.900000,132.250000,4.580750,3878.750000
50%,1186.550000,1196.500000,1180.150000,1187.450000,1188.000000,1188.150000,214.000000,8.190500,6932.500000
75%,1237.225000,1243.000000,1231.425000,1238.600000,1237.475000,1237.475000,306.000000,14.731000,12425.000000
max,1346.800000,1350.000000,1312.000000,1340.000000,1345.800000,1345.800000,1371.000000,140.592000,110281.000000
